# Topic modelling of news headlines for prediction of news category¶

Now we wan't to perform unsupervised learning and train a modell for prediction of news category labels without using the provided labels in the corpus.
We use the LDA classifier and assume 6 topics. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load data

In [2]:
train = pd.read_csv('../../preprocessing/train.csv')
test = pd.read_csv('../../preprocessing/test.csv')

In [3]:
train.head()

,date,title,description,category,text,source,frequent_category
0,2023-06-29 11:57:00,Cyberbetrug in Philippinen: Großrazzia gegen M...,In den Philippinen wurden 2.724 Personen festg...,Missing,NaN,taz,Missing
1,2023-01-15 09:16:00,"GPT: Eine Software, die die Welt verändert","GPT-3 ist eine jener Entwicklungen, die binnen...",Terra X - die Wissens-Kolumne,NaN,ZDF heute,Other
2,2023-03-04 15:13:32,Fußball: Hollerbach schießt Wehen Wiesbaden mi...,NaN,News,NaN,Zeit,News
3,2022-07-16 07:17:44,Leichtathletik-WM: Ein Auftakt mit Licht und S...,"Stabhochspringerin Jacqueline Otchere jubelte,...",Missing,NaN,Tagesschau,Missing
4,2022-10-11 18:49:00,Geflüchtete in Deutschland: Nicht alle gleich ...,Die Innenministerin spricht sich für Solidarit...,Missing,NaN,taz,Missing


In [4]:
test.head()

,date,title,description,category,text,source,frequent_category
0,2022-07-15 09:40:00,Lauterbach für vierte Corona-Impfung: Zweiter ...,Wegen steigender Coronazahlen rät Gesundheitsm...,Missing,NaN,taz,Missing
1,2023-06-15 22:08:22,Weyrauch zieht Bewerbung um rbb-Intendantenste...,Einen Tag vor der Intendantenwahl beim rbb hat...,Missing,NaN,Tagesschau,Missing
2,2022-08-20 06:29:28,"Inflation in Großbritannien: ""Es wird alles te...",Die Energiepreise: steigen und steigen. Die In...,Missing,NaN,Tagesschau,Missing
3,2023-02-12 22:14:58,Nahe kanadischer Grenze - Schon wieder! USA sc...,Medienberichten zufolge haben die USA erneut e...,Ausland,NaN,Focus,Ausland
4,2022-08-26 11:50:37,China: Geschichte verdeht: Mao wollte Taiwan a...,Chinas Kommunistische Partei bedroht Taiwan mi...,"China, Taiwan, Kommunistische Partei, Peking, ...",NaN,stern,Other


## Load German stopwords

In [5]:
stop_words = pd.read_csv('german_stopwords.txt', header=None)[0].values.tolist()

In [6]:
print(stop_words)

['ab', 'aber', 'alle', 'allein', 'allem', 'allen', 'aller', 'allerdings', 'allerlei', 'alles', 'allmählich', 'allzu', 'als', 'alsbald', 'also', 'am', 'an', 'and', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'andererseits', 'anderes', 'anderm', 'andern', 'andernfalls', 'anders', 'anstatt', 'auch', 'auf', 'aus', 'ausgenommen', 'ausser', 'ausserdem', 'außer', 'außerdem', 'außerhalb', 'bald', 'bei', 'beide', 'beiden', 'beiderlei', 'beides', 'beim', 'beinahe', 'bereits', 'besonders', 'besser', 'beträchtlich', 'bevor', 'bezüglich', 'bin', 'bis', 'bisher', 'bislang', 'bist', 'bloß', 'bsp.', 'bzw', 'ca', 'ca.', 'content', 'da', 'dabei', 'dadurch', 'dafür', 'dagegen', 'daher', 'dahin', 'damals', 'damit', 'danach', 'daneben', 'dann', 'daran', 'darauf', 'daraus', 'darin', 'darum', 'darunter', 'darüber', 'darüberhinaus', 'das', 'dass', 'dasselbe', 'davon', 'davor', 'dazu', 'daß', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'dem', 'demnach', 'demselben', 'den', 'denen', 'denn',

## Extract features from 'title' 

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words=stop_words)

In [9]:
# create Document-Term-Matrix
dtm = cv.fit_transform(train['title'])

C:\Users\manuela.rink\anaconda3\envs\nlp\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['bsp', 'zb'] not in stop_words.
  warnings.warn(


In [10]:
dtm

<53531x21449 sparse matrix of type '<class 'numpy.int64'>'
	with 274945 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.decomposition import LatentDirichletAllocation

In [12]:
lda = LatentDirichletAllocation(n_components=6, random_state=42)
lda.fit(dtm)

LatentDirichletAllocation(n_components=6, random_state=42)

## Analyse extracted features

In [13]:
len(cv.get_feature_names_out())

21449

In [14]:
cv.get_feature_names_out()

array(['00', '000', '007', ..., 'übung', 'übungen', 'украинцы'],
      dtype=object)

In [15]:
len(lda.components_)

6

In [16]:
lda.components_

array([[  0.16684159,  95.44342752,   0.1666677 , ...,   4.01039289,
          0.16713425,   0.16667149],
       [  0.16666733,  67.74460047,   0.16666754, ...,   3.81209869,
          2.28654669,   0.16667097],
       [  0.1671256 , 109.12864667,   0.16828477, ...,   0.16959585,
          1.2718277 ,   0.16667232],
       [  2.16584595,   0.16760458,   0.16667342, ...,   3.67334276,
          2.62498034,   0.16667216],
       [  0.16666757,   0.70861897,   2.16503884, ...,   0.16712812,
          1.48276022,   0.16667257],
       [  0.16685196,  68.80710178,   0.16666773, ...,   0.16744169,
          0.1667508 ,   2.16664048]])

In [17]:
len(lda.components_[0])

21449

### Show most important words of first extracted topic

In [18]:
first_topic = lda.components_[0]

In [19]:
first_topic.argsort() # returns indices of ascending sorted array values 

array([10926, 11727,  4581, ...,  5537, 20137,  5482], dtype=int64)

In [20]:
# the last 10 entries are the 10 most important words for topic 1
top_words_indices = first_topic.argsort()[-10:]

In [21]:
for index in top_words_indices:
    print(cv.get_feature_names_out()[index])

deutschland
haft
us
frankreich
scholz
trump
eu
ex
wegen
euro


### Show most important words of first extracted topic

In [22]:
for index, topic in enumerate(lda.components_):
    print(f'Die TOP-15 Wörter für das Thema #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

Die TOP-15 Wörter für das Thema #0
['grünen', 'ticket', 'jahre', 'präsident', 'faeser', 'deutschland', 'haft', 'us', 'frankreich', 'scholz', 'trump', 'eu', 'ex', 'wegen', 'euro']


Die TOP-15 Wörter für das Thema #1
['angriff', 'russischer', 'selenskyj', 'neue', 'invasion', 'stimmen', 'tote', 'iran', 'putins', 'lage', 'putin', 'russische', 'russland', 'krieg', 'ukraine']


Die TOP-15 Wörter für das Thema #2
['analyse', 'tote', 'putin', 'moskau', 'kiew', 'g7', 'zwei', 'us', 'taiwan', 'baerbock', 'gipfel', 'deutschland', 'usa', 'scholz', 'china']


Die TOP-15 Wörter für das Thema #3
['bundesliga', 'tot', 'geht', 'frauen', 'generation', 'letzte', 'türkei', 'us', 'wahrheit', 'cdu', 'wahl', 'wm', 'marktbericht', 'afd', 'berlin']


Die TOP-15 Wörter für das Thema #4
['türkei', 'stream', 'ukraine', 'eu', 'nord', 'chef', 'us', 'putin', 'russland', 'fordert', 'gas', 'akw', 'israel', 'deutschland', 'biden']


Die TOP-15 Wörter für das Thema #5
['regierung', 'news', 'deutschen', 'inflation', 'lau

### Get topics with highest probability for news items in train data

In [23]:
topic_results = lda.transform(dtm)
topic_results.shape

(53531, 6)

In [24]:
topic_results[0].round(5)

array([0.04176, 0.04172, 0.79136, 0.04172, 0.04175, 0.04169])

In [25]:
topic_results[0].argmax()

2

In [26]:
# save topic to train dataset
train['topic'] = topic_results.argmax(axis=1)

In [27]:
train.head()

,date,title,description,category,text,source,frequent_category,topic
0,2023-06-29 11:57:00,Cyberbetrug in Philippinen: Großrazzia gegen M...,In den Philippinen wurden 2.724 Personen festg...,Missing,NaN,taz,Missing,2
1,2023-01-15 09:16:00,"GPT: Eine Software, die die Welt verändert","GPT-3 ist eine jener Entwicklungen, die binnen...",Terra X - die Wissens-Kolumne,NaN,ZDF heute,Other,2
2,2023-03-04 15:13:32,Fußball: Hollerbach schießt Wehen Wiesbaden mi...,NaN,News,NaN,Zeit,News,5
3,2022-07-16 07:17:44,Leichtathletik-WM: Ein Auftakt mit Licht und S...,"Stabhochspringerin Jacqueline Otchere jubelte,...",Missing,NaN,Tagesschau,Missing,3
4,2022-10-11 18:49:00,Geflüchtete in Deutschland: Nicht alle gleich ...,Die Innenministerin spricht sich für Solidarit...,Missing,NaN,taz,Missing,4


In [28]:
#Save result to csv
train.to_csv('train_lda_topics_6.csv')